### Navigation
1. [Day 11](#Day-11)  
1. [Day 12](#Day-12)  
1. [Day 13](#Day-13)  
1. [Day 14](#Day-14)  
1. [Day 15](#Day-15)   
1. [Day 16](#Day-16)  
1. [Day 17](#Day-17)  
1. [Day 18](#Day-18)  
1. [Day 19](#Day-19)  
1. [Day 20](#Day-20)  

# Day 11
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/11

### --- Part One ---

In [1]:
import numpy as np

with open("data/day11-input1.txt", 'r') as file:
    inputs = [line.replace('\n', '') for line in file.readlines()]
    inputs = np.asarray([[int(c) for c in r] for r in inputs])
    inputs = np.pad(inputs, [(1,1),(1,1)], constant_values=11)

In [2]:
inputs

array([[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11],
       [11,  1,  2,  2,  4,  3,  4,  6,  3,  8,  4, 11],
       [11,  5,  6,  2,  1,  1,  2,  8,  5,  8,  7, 11],
       [11,  6,  3,  8,  8,  4,  2,  6,  5,  4,  6, 11],
       [11,  1,  5,  5,  6,  2,  4,  7,  7,  5,  6, 11],
       [11,  1,  4,  5,  1,  8,  1,  1,  5,  7,  3, 11],
       [11,  1,  8,  3,  2,  3,  8,  8,  1,  2,  2, 11],
       [11,  2,  7,  4,  8,  5,  4,  5,  6,  4,  7, 11],
       [11,  2,  5,  8,  2,  8,  7,  7,  4,  3,  2, 11],
       [11,  3,  1,  8,  5,  6,  4,  3,  8,  7,  1, 11],
       [11,  2,  2,  2,  4,  8,  7,  6,  6,  2,  7, 11],
       [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]])

In [3]:
def recursion(inputs, zero_points=[]):
    # get all with power 10
    power10 = np.where(inputs==10)
    power10_pts = list(zip(power10[0], power10[1]))
    # move all the 10s to 11s so that we dont count them anymore
    for pt in power10_pts:
        inputs[pt] += 1
    zero_points += power10_pts
    for r, c in power10_pts:
        inputs[r-1:r+2, c-1:c+2] += 1
        inputs, zero_points = recursion(inputs, zero_points)
    return inputs, zero_points

In [4]:
flash_counter = 0
i = 0
while(True):
    i += 1
    # increase power by 1
    inputs += 1
    inputs, zero_points = recursion(inputs, zero_points=[])
    # increase flash counter
    flash_counter += len(zero_points)
    # replace flashed points with zeros
    for pt in zero_points:
        inputs[pt] = 0
    if i == 100:
        print("Answer for part one:", flash_counter)
    if len(zero_points) == 100:
        print("Answer for part two:", i)
        break
    

Answer for part one: 1591
Answer for part two: 314


# Day 12
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/12

### --- Part One ---

In [5]:
with open("data/day12-input1.txt", 'r') as file:
    inputs = [line.replace('\n', '') for line in file.readlines()]

In [6]:
from collections import Counter

def add_to_graph(graph, a, b):
    if a in graph.keys():
        graph[a] += [b]
    else:
        graph[a] = [b]
    return graph

def check_revisit_possible(path):
    path = [cave for cave in path
            if str.islower(cave) and cave not in ['start', 'end']]
    counts = Counter(path)
    if sum(counts.values()) == len(counts.keys()):
        return True
    return False

def count_paths(curr_paths, graph, paths_counter, part_two=True):
    for curr_path in curr_paths:
        last_position = curr_path[-1]
        posible_moves = graph[last_position]
        if part_two:
            revisit = check_revisit_possible(curr_path)
        for pos_move in posible_moves:
            if pos_move == 'end':
                paths_counter += 1
            elif (str.isupper(pos_move) or pos_move not in curr_path
                  or (part_two
                      and pos_move != 'start'
                      and revisit)):
                # recursion
                paths_counter = count_paths(
                    [curr_path + [pos_move]], graph, paths_counter, part_two)
    return paths_counter

In [7]:
graph = {}
for mapping in inputs:
    a, b = mapping.split('-')
    add_to_graph(graph, a, b)
    add_to_graph(graph, b, a)

In [8]:
%%time
all_paths = count_paths([['start']], graph, 0, False)
all_paths

Wall time: 9.06 ms


3802

### --- Part Two ---

In [9]:
%%time
all_paths = count_paths([['start']], graph, 0, True)
all_paths

Wall time: 1.48 s


99448

# Day 13
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/13

### --- Part One ---

In [10]:
with open("data/day13-input1.txt", 'r') as file:
    inputs = [line.replace('\n', '') for line in file.readlines() if line != '\n']
    dots = [[int(v) for v in line.split(',')] for line in inputs if ',' in line]
    instructions = [line.split('=') for line in inputs if '=' in line]

In [11]:
import numpy as np

dots_np = np.asarray(dots)

x_max = np.asarray(dots_np)[:,0].max() + 1
y_max = np.asarray(dots_np)[:,1].max() + 1

x_max = x_max + 1 if x_max % 2 == 0 else x_max
y_max = y_max + 1 if y_max % 2 == 0 else y_max

dots_paper_np = np.zeros((y_max, x_max))
for x, y in dots_np:
    dots_paper_np[y, x] = 1

In [12]:
def fold_paper(paper, instr, v):
    if 'x' in instr:
        paper_a = paper[:,:v]
        paper_b = np.fliplr(paper[:,v+1:])
    elif 'y' in instr:
        paper_a = paper[:v,:]
        paper_b = np.flipud(paper[v+1:,:])
    return np.logical_or(paper_a, paper_b)

In [13]:
folded_paper = dots_paper_np.copy()
for i, instr in enumerate(instructions[:]):
    instr, v = instr
    folded_paper = fold_paper(folded_paper, instr, int(v))
    if i == 0:
        print("Answer part one:",folded_paper.sum())

Answer part one: 737


### --- Part Two ---

In [14]:
for i, v in enumerate(range(4,40,5)):
    print("---Letter ", i+1, "---")
    print(folded_paper[:,v-4:v] * 1)

---Letter  1 ---
[[1 1 1 1]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 1 1 1]]
---Letter  2 ---
[[1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [0 1 1 0]]
---Letter  3 ---
[[0 0 1 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [1 0 0 1]
 [0 1 1 0]]
---Letter  4 ---
[[1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]
 [0 1 1 0]]
---Letter  5 ---
[[0 1 1 0]
 [1 0 0 1]
 [1 0 0 1]
 [1 1 1 1]
 [1 0 0 1]
 [1 0 0 1]]
---Letter  6 ---
[[1 1 1 1]
 [1 0 0 0]
 [1 1 1 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]]
---Letter  7 ---
[[1 0 0 1]
 [1 0 0 1]
 [1 1 1 1]
 [1 0 0 1]
 [1 0 0 1]
 [1 0 0 1]]
---Letter  8 ---
[[1 1 1 0]
 [1 0 0 1]
 [1 0 0 1]
 [1 1 1 0]
 [1 0 0 0]
 [1 0 0 0]]


# Day 14
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/14

### --- Part One ---

In [15]:
with open("data/day14-input1.txt", 'r') as file:
    inputs_raw = file.read()
    inputs = [line for line in inputs_raw.split('\n') if line != '']
    instructions = [line.replace(' ', '').split('->')
                    for line in inputs if '>' in line]

This is an example of template breakdown I'm trying to implement with my code
```
Template:     NNC
              NN NC
After step 1: NCN NBC
              NC CN NB BC
After step 2: NBC CCN NBB BBC
              NB BC CC CN NB BB BB BC
After step 3: NBB BBC CNC CCN NBB BNB BNB BBC
              NB BB BB BC CN NC CC CN NB BB BN NB BN NB BB BC
```
At the end what I get is char pairs counter dictionary.  
I then break down this pairs counter into final per-char counter.  
Because of how the breakdown is made I end up with duplicate counts.  
Only characters that are not duplicated are the edge chars from the template ('N' and 'C')

In [16]:
def calc_rounds(rounds, template, instructions):
    formula_mapping = dict(instructions)
    # formula_mapping = {'FK': 'O', 'BK': 'B', 'PB': 'N', 'VS': 'P', ...}
    pairs_counter = dict(
        [(k, 0) for k in formula_mapping.keys()]
    )
    # pairs_counter = {'FK': 0, 'BK': 0, 'PB': 0, 'VS': 0, 'OF': 0, ...}
    # slice input template chars into a set of pairs
    for i in range(len(template)-1):
        # and count occurances of each pair
        pairs_counter[template[i:i+2]] += 1
    # iterate rounds
    for i in range(rounds):
        counter_pairs = pairs_counter.copy().items()
        # iterate through keys and values of pairs_counter
        for k, v in counter_pairs:
            # engage only if counter indicates given pair exist in current template
            if v > 0:
                # for given pair (k) find a char to be inserted between k[0] and k[1]
                # ex: k == 'CN' and r == 'B'
                r = formula_mapping[k]
                # build new pairs and copy occurances from k pair
                # ex: k[0] + r == 'CB'
                #     r + k[1] == 'BN'
                pairs_counter[k[0] + r] += v
                pairs_counter[r + k[1]] += v
                # since we broke this pair into new pairs
                # decrease occurances for old pair
                pairs_counter[k] -= v
    # calc score
    return calculate_score(
        formula_mapping,
        pairs_counter,
        edge_chars=[template[0], template[-1]])

def calculate_score(formula_mapping, pairs_counter, edge_chars):
    # formula_mapping = {'FK': 'O', 'BK': 'B', 'PB': 'N', 'VS': 'P', ...}
    # pairs_counter = {'FK': 43, 'BK': 799, 'PB': 163, 'VS': 1, 'OF': 161, ...}
    final_counter = {}
    for c in ''.join(formula_mapping.keys()):
        final_counter[c] = 0
    for c, v in [(c, i[1]) for i in pairs_counter.items()
                 for c in i[0]]:
        final_counter[c] += v
    # final_counter = {'F': 2494, 'K': 6204, 'B': 6774, 'P': 1652, 'V': 1288, ...}
    
    # find characters with max and min number of values
    max_k = max(final_counter, key=final_counter.get)
    min_k = min(final_counter, key=final_counter.get)
    # subtract value for min_k from max_k to create a score
    score = final_counter[max_k] - final_counter[min_k]
    # edge chars from the template where not duplicated
    # so we need to add / substract them from the score respectively
    score = score + 1 if max_k in edge_chars else score
    score = score - 1 if min_k in edge_chars else score
    # floor the final score if needed and return it
    # for the final count values we need to divide each by 2
    # because we made duplicates when breaking down original template into pairs
    return int(score / 2)

In [17]:
%%time
calc_rounds(10, inputs[0], instructions)

Wall time: 998 µs


3306

### --- Part Two ---

In [18]:
%%time
calc_rounds(40, inputs[0], instructions)

Wall time: 1.99 ms


3760312702877

# Day 15
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/15

### --- Part One ---

In [19]:
inputs = """1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"""

In [20]:
import numpy as np

with open("data/day15-input1.txt", 'r') as file:
    inputs = file.read()

In [21]:
inputs = [line for line in inputs.split('\n')]
inputs = np.asarray([[int(i) for i in line] for line in inputs])

In [22]:
import numpy as np

def min_risk(risk_map, r, c):
    risk_map = np.asarray(risk_map)
    # total risk matrix
    tr = np.zeros_like(risk_map)
    rang = list(range(1, r+1))
 
    # Initialize first row of tc array
    # because it's unaffected by adjacent cells
    for j in rang:
        tr[0, j] = tr[0, j-1] + risk_map[0, j]
    # Initialize first column of total cost(tc) array
    # because it's unaffected by adjacent cells
    for i in rang:
        tr[i, 0] = tr[i-1, 0] + risk_map[i, 0]
 
    # Construct rest of the tr array
    for i in rang:
        for j in rang:
            tr[i, j] = min(tr[i-1, j], tr[i, j-1]) + risk_map[i, j]

    return tr[r,c]

In [23]:
min_risk(inputs, len(inputs)-1, len(inputs)-1)

390

### --- Part Two ---

In [24]:
# construct larger cave

size = len(inputs)
M = np.zeros((size * 5, size * 5))
M[:size, :size] = inputs

for c in range(1, 5):
    M[:size, size*c:size*(c+1)] = M[:size, size*(c-1):size*c] + 1

for c in range(0, 5):
    for r in range(1, 5):
        M[size*r:size*(r+1), size*c:size*(c+1)] = M[size*(r-1):size*r, size*c:size*(c+1)] + 1

# np.where faster than M > 9, 4.40ms vs 6.40ms
M = np.where(M > 9, M - 9, M)
# mask = M > 9
# M[mask] -= 9

In [25]:
from queue import PriorityQueue
from collections import defaultdict
        
def dijkstra(risk_map, start_vertex=(0, 0)):
    r_max, c_max = len(risk_map), len(risk_map[0])
    visited = set()
    D = {}
    D[start_vertex] = 0

    pq = PriorityQueue()
    pq.put((0, start_vertex))

    while not pq.empty():
        (dist, current_vertex) = pq.get()
        visited.add(current_vertex)

        r, c = current_vertex

        neighbors = set()
        if r > 0: neighbors.add((r-1, c))
        if r < r_max-1: neighbors.add((r+1, c))        
        if c > 0: neighbors.add((r, c-1))
        if c < c_max-1: neighbors.add((r, c+1))

        for neighbor in neighbors:
            distance = risk_map[neighbor]
            if neighbor not in visited:
                old_cost = D[neighbor] if neighbor in D.keys() else float('inf')
                new_cost = D[current_vertex] + distance
                if new_cost < old_cost:
                    pq.put((new_cost, neighbor))
                    D[neighbor] = new_cost
    return D

In [26]:
%%time

r_max, c_max = len(M), len(M[0])

D = dijkstra(M)
print(D[(r_max-1, c_max-1)])

2814.0
Wall time: 2.59 s


# Day 16
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/16

In [27]:
import numpy as np

with open("data/day16-input.txt", 'r') as file:
    inputs = file.read()

In [28]:
from functools import reduce

packets_dict = {
    0: sum,
    1: lambda l: reduce((lambda x, y: x * y), l),
    2: min,
    3: max,
    5: lambda l: 1 if l[0] > l[1] else 0,
    6: lambda l: 1 if l[0] < l[1] else 0,
    7: lambda l: 1 if l[0] == l[1] else 0
}

def hex_to_bin(hex_string):
    return ''.join(
        [bin(int(h, 16))[2:].zfill(4) for h in hex_string]
    )

def get_v_and_pid(bits, idx):
    # get packet version, bin->int
    v = int(bits[idx:idx+3], 2)
    # packet id, bin->int
    pid = int(bits[idx+3:idx+6], 2)
    idx += 6
    return idx, v, pid

In [29]:
def decode_packet(
        bits, idx, 
        versions_list,
        subpackets_num = None):
    values_list = []

    while True:
        if idx >= len(bits):
            break
        if len(bits[idx:]) < 11:
            break
        if subpackets_num is not None and subpackets_num == 0:
            break
        
        idx, v, pid = get_v_and_pid(bits, idx)
        versions_list.append(v)
        if pid == 4:
            num = ''
            while True:
                temp_num = bits[idx:idx+5]
                num += temp_num[1:]
                idx += 5
                if temp_num[0] == '0':
                    break
            num = int(num, 2)
            values_list.append(num)
        else:
            if bits[idx] == '0':
                subpackets_length = int(bits[idx+1:idx+1+15], 2)
                idx += 1 + 15
                _, versions_list, values_list_temp = decode_packet(
                    bits[idx: idx + subpackets_length], 0, versions_list)
                idx += subpackets_length
            elif bits[idx] == '1':
                subpackets_num_temp = int(bits[idx+1:idx+1+11], 2)
                idx += 1 + 11
                idx, versions_list, values_list_temp = decode_packet(
                    bits, idx, versions_list, subpackets_num_temp)
            values_list.append(packets_dict[pid](values_list_temp))
            
        subpackets_num = subpackets_num - 1 if subpackets_num is not None else None

    return idx, versions_list, values_list

In [30]:
bits = hex_to_bin(inputs)
_, versions, values_list = decode_packet(bits, 0, [])

In [31]:
print("Part 1 answer:", sum(versions))
print("Part 2 answer:", values_list[0])

Part 1 answer: 963
Part 2 answer: 1549026292886


# Day 17
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/17

### --- Part One ---

In [32]:
inputs = "target area: x=20..30, y=-10..-5"

In [33]:
import numpy as np

with open("data/day17-input1.txt", 'r') as file:
    inputs = file.read()

In [34]:
targets = inputs.replace(' ', '').split(':')[1].split(',')
print(targets)
targets = [[int(v) for v in target.split('=')[1].split('..')] for target in targets]

['x=209..238', 'y=-86..-59']


In [35]:
x_target_range = list(range(targets[0][0], targets[0][1] + 1))
y_target_range = list(range(targets[1][0], targets[1][1] + 1))
y_target_range.reverse()
print("X target:", x_target_range)
print("Y target:", y_target_range)
x_target_min, x_target_max = x_target_range[0], x_target_range[-1]
y_target_min, y_target_max = y_target_range[0], y_target_range[-1]

X target: [209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238]
Y target: [-59, -60, -61, -62, -63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79, -80, -81, -82, -83, -84, -85, -86]


In [36]:
def step(s, v, y_high=0):
    s = [sum(x) for x in zip(s, v)]
    v[1] -= 1
    if v[1] == 0:
        y_high = s[1]

    if v[0] > 0:
        v[0] -= 1
    elif v[0] < 0:
        v[0] += 1
    
    if s[0] in x_target_range and s[1] in y_target_range:
        return True, y_high
    if s[0] > x_target_max or s[1] < y_target_max:
        return False, 0
    
    return step(s, v, y_high)

In [37]:
y_high_max = 0
v_list = set()

for vx in range(0, x_target_max + 1):
    for vy in range(y_target_max, 90):
        on_target, y_high = step((0,0), [vx, vy])
        if on_target:
            v_list.add((vx, vy))
            if y_high > y_high_max:
                y_high_max = y_high

In [38]:
y_high_max

3655

### --- Part Two ---

In [39]:
len(v_list)

1447

# Day 18
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/18

### --- Part One ---

In [40]:
import numpy as np

with open("data/day18-input.txt", 'r') as file:
    inputs = file.read()

In [41]:
import copy


def explode(s_in):
    s = [c for c in s_in]
    level_count = 0
    l_reg = ''
    l_reg_i = None
    r_reg = ''
    exploded = False
    collect = False
    pair = ''
    i = 0
    i_max = len(s)
    while True:
        if level_count == 4 and s[i] == '[' and not exploded:
            collect = True
        if collect:
            j = i
            while j >= 0:
                j -= 1
                if s[j] not in [',', '[', ']']:
                    l_reg = s[j] + l_reg
                    s[j] = 'L'
                elif s[j+1] == 'L':
                    break
            while True:
                pair += s[i]
                s[i] = 'P'
                i += 1
                if pair[-1] == ']':
                    break
            collect = False
            exploded = True
        elif s[i] == '[':
            level_count += 1
        elif s[i] == ']':
            level_count -= 1
        elif s[i] not in [',', '[', ']'] and exploded:
            while True:
                r_reg += s[i]
                s[i] = 'R'
                i += 1
                if s[i] in [',', '[', ']']:
                    break
            break

        i += 1
        if i >= i_max:
            break

    s = ''.join(s)
    if pair != '':
        # do the math
        pair_length = len(pair)
        pair = [int(v) for v in pair.replace('[', '').replace(']', '').split(',')]
        # replace left_regular with left most value from the pair
        if l_reg != '':
            s = s.replace(
                ''.join(['L' for p in l_reg]),
                str((int(l_reg) + pair[0])))
        # replace right_regular with left most value from the pair
        if r_reg != '':
            s = s.replace(
                ''.join(['R' for p in r_reg]),
                str((int(r_reg) + pair[1])))
        # replace exploded pair with 0
        s = s.replace(
            ''.join(['P' for p in range(pair_length)]),
            '0')
        
        s_in = s
    return s_in


def split(s_in):
    if type(s_in) != list and s_in >= 10:
        s_in = [int(np.floor(s_in/2)), int(np.ceil(s_in/2))]
        return s_in
    elif type(s_in) == list:
        for i, s in enumerate(s_in):
            s1 = split(copy.copy(s))
            if str(s1) != str(s):
                s_in[i] = s1
                break
    return s_in


def calc_magnitude(s_in, idx=None):
    if type(s_in) == list:
        for i, s in enumerate(s_in):
            s = calc_magnitude(s, i)
            s_in[i] = s
        s_in = sum(s_in)
    if idx is not None:
        if idx == 0:
            return s_in * 3
        if idx == 1:
            return s_in * 2
    return s_in    

In [42]:
import ast

# snail_numbers = [ast.literal_eval(v) for v in inputs.split('\n')]
snail_numbers = inputs.split('\n')

while True:
    a, b = snail_numbers[0], snail_numbers[1]
    # addition
    c = '[%s,%s]' % (a, b)
    
    while True:
        # check explode
        c2 = explode(c)
        if c != c2:
            c = c2
            continue

        # check split
        c2 = split(ast.literal_eval(c))
        c2 = str(c2).replace(' ', '')
        if c2 != c:
            c = c2
            continue
        break
    snail_numbers[0] = c
    snail_numbers.pop(1)
    if len(snail_numbers) < 2:
        break
        
calc_magnitude(ast.literal_eval(snail_numbers[0]))

3359

### --- Part Two ---

In [43]:
import ast
import itertools

snail_numbers = inputs.split('\n')

pairs_list = list(itertools.product(
    range(len(snail_numbers)),
    range(len(snail_numbers))))

list_magnitudes = set()

for x,y in pairs_list:
    a, b = snail_numbers[x], snail_numbers[y]
    # addition
    c = '[%s,%s]' % (a, b)
    while True:
        # check explode
        c2 = explode(c)
        if c != c2:
            c = c2
            continue
        # check split
        c2 = split(ast.literal_eval(c))
        c2 = str(c2).replace(' ', '')
        if c2 != c:
            c = c2
            continue
        break
    list_magnitudes.add(calc_magnitude(ast.literal_eval(c)))
    
max(list_magnitudes)

4616

# Day 19
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/19

### --- Part One ---

In [44]:
import numpy as np

with open("data/day19-input1.txt", 'r') as file:
    inputs = file.read()

In [45]:
inputs  = [np.asarray([[int(xyz) for xyz in coords.split(',')]
            for coords in scanner.split('\n')[1:]])
           for scanner in inputs.split('\n\n')]

In [46]:
def generate_xy(length, half=False):
    x1, y1 = np.triu_indices(length, 1)
    x2, y2 = np.tril_indices(length, -1)
    x = np.concatenate((x1, x2), axis=0)
    y = np.concatenate((y1, y2), axis=0)
    if half:
        return x1, y1
    return x, y

def row_wise_abs_subtract(arr, half=False):
    x, y = generate_xy(len(arr), half)
    out_arr = np.abs(arr[x] - arr[y])
    return out_arr.tolist()

def get_counts_dict(beacons):
    uniques = np.unique(beacons, return_counts=True, axis=0)
    counts = uniques[1]
    counts_dict = dict(zip([tuple(k.tolist()) for k in uniques[0]], counts))
    return counts_dict
    

In [47]:
from collections import Counter

beacons_per_scanner_count = [len(scanner) for scanner in inputs]
# print(beacons_per_scanner_count)
scanners = [row_wise_abs_subtract(scanner) for scanner in inputs]
dist_per_scanner_count = [len(scanner) for scanner in scanners]
scanners_sorted = [[sorted(beacon) for beacon in beacons] for beacons in scanners]

beacons_sorted = [tuple(beacon) for scanner in scanners_sorted
                  for beacon in scanner]
counts_dict = get_counts_dict(beacons_sorted)

total_counter = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0}

current_i = 0

for beacons_num, num_dists in zip(beacons_per_scanner_count, dist_per_scanner_count):
    x, y = generate_xy(beacons_num)
    matrix = np.zeros((beacons_num, beacons_num))
    temp_beacons = beacons_sorted[current_i: current_i+num_dists]
    temp_counts = [counts_dict[beacon] for beacon in temp_beacons]
#     print(len(temp_counts), len(list(zip(x,y))))

    for j, xy in enumerate(zip(x,y)):
        matrix[xy] = temp_counts[j]/2 - 1 # duplicated so 2,4,6,8,10 -> 0,1,2,3,4
    
    counter = Counter(np.max(matrix, axis=0))
    for k,v in counter.items():
        total_counter[k] += v
#     print(counter)
    current_i += num_dists

total_beacons = 0
for k in total_counter.keys():
    total_beacons += total_counter[k]/(k+1)
print(total_beacons)

408.0


### --- Part Two ---

In [48]:
#!/usr/bin/python
# src: https://github.com/nghiaho12/rigid_transform_3D
# and really nice ang thorough read in this topic: 
# https://towardsdatascience.com/understanding-singular-value-decomposition-and-its-application-in-data-science-388a54be95d
import numpy as np

# Input: expects 3xN matrix of points
# Returns R,t
# R = 3x3 rotation matrix
# t = 3x1 column vector

def rigid_transform_3D(A, B):
    assert A.shape == B.shape

    num_rows, num_cols = A.shape
    if num_rows != 3:
        raise Exception(f"matrix A is not 3xN, it is {num_rows}x{num_cols}")

    num_rows, num_cols = B.shape
    if num_rows != 3:
        raise Exception(f"matrix B is not 3xN, it is {num_rows}x{num_cols}")

    # find mean column wise
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)

    # ensure centroids are 3x1
    centroid_A = centroid_A.reshape(-1, 1)
    centroid_B = centroid_B.reshape(-1, 1)

    # subtract mean
    Am = A - centroid_A
    Bm = B - centroid_B
    
    # matrix multiplication Am x Bm.T
    H = Am @ np.transpose(Bm)

    # sanity check
    #if linalg.matrix_rank(H) < 3:
    #    raise ValueError("rank of H = {}, expecting 3".format(linalg.matrix_rank(H)))

    # find rotation using Singular Value Decomposition
    U, S, Vt = np.linalg.svd(H)
    # rotation matrix [3x3]
    R = Vt.T @ U.T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("det(R) < R, reflection detected!, correcting for it ...")
        Vt[2,:] *= -1
        R = Vt.T @ U.T
    
    # translation vector [3x1]
    t = -R @ centroid_A + centroid_B
#     return R, t

    return np.round(R), np.round(t)

In [49]:
%%time
import itertools
from collections import defaultdict

def crack_beacons_alignment(idices_A, idices_B):
    k_count = max(max(idices_A)) + 1
    mapping = {}
    while k_count > len(mapping.keys()):
        for i in range(1, len(idices_A)):
            ia_p = idices_A[i-1]
            ia = idices_A[i]
            ib_p = idices_B[i-1]
            ib = idices_B[i]
            if ia[0] in ia_p:
                mapping[ia[0]] = ib[0] if ib[0] in ib_p else ib[1]
                mapping[ia[1]] = ib[0] if ib[0] not in ib_p else ib[1]
            elif ia[1] in ia_p:
                mapping[ia[1]] = ib[0] if ib[0] in ib_p else ib[1]
                mapping[ia[0]] = ib[0] if ib[0] not in ib_p else ib[1]
            elif ia[1] in mapping.keys():
                mapping[ia[0]] = ib[0] if ib[1] == mapping[ia[1]] else ib[1]
            elif ia[0] in mapping.keys():
                mapping[ia[1]] = ib[0] if ib[1] == mapping[ia[0]] else ib[1]
    
    return mapping

correlation_map = defaultdict(list)
transformation_map = defaultdict(list)

# iterator = list(itertools.product(range(len(scanners)), range(len(scanners))))
x, y = np.triu_indices(len(scanners), 1)
all_examples = []
all_examples2 = []
all_transforms = []
all_xy =[]
for x, y in zip(x, y):
    beacons_ab = np.concatenate((scanners_sorted[x], scanners_sorted[y]), axis=0)
    uniques = np.unique(beacons_ab, return_counts=True, axis=0)
    dupes_mask = uniques[1] == 4
    sum_dupes = sum(dupes_mask)
    # do the magic only if we have exactly 12 matching beacons between scanners
    if sum_dupes == 66:
        matrices = []
        for idx in [x, y]:
            temp_dict = dict([(tuple(u.tolist()), 0) for u in uniques[0]])
            temp_dict.update(dict([(tuple(u.tolist()), 1) for u in uniques[0][dupes_mask]]))
            temp_counts = [temp_dict[tuple(b)] for b in scanners_sorted[idx]]
            beacons_num = beacons_per_scanner_count[idx]
            matrix = np.zeros((beacons_num, beacons_num))
            a, b = generate_xy(beacons_num)
            
            for j, xy in enumerate(zip(a, b)):
                matrix[xy] = temp_counts[j]
            matrices.append(matrix.max(axis=1))
            
        A = inputs[x][matrices[0]==1]
        B = inputs[y][matrices[1]==1]
        ### aligning A beacons with B beacons ###
        list_A_dist = [dist for dist in row_wise_abs_subtract(A, half=True)]
        list_B_dist = [dist for dist in row_wise_abs_subtract(B, half=True)]
        list_A = [sum(dist) for dist in list_A_dist]
        list_B = [sum(dist) for dist in list_B_dist]
        ai, bi = generate_xy(len(A), half=True)
        sorting_A = np.argsort(list_A)
        sorting_B = np.argsort(list_B)
        idices_A = list(zip(ai[sorting_A], bi[sorting_A]))
        idices_B = list(zip(ai[sorting_B], bi[sorting_B]))        
        beacons_mapping = crack_beacons_alignment(idices_A, idices_B)
        B = B[[beacons_mapping[i] for i in range(len(A))]]
        ###
        ### getting transformation matrix for each pair
        ret_R1, ret_t1 = rigid_transform_3D(A[:].T, B[:].T)
        ret_R2, ret_t2 = rigid_transform_3D(B[:].T, A[:].T)
        ###
        ### validate correctness and save
        ### apparently there are some points which are breaking my code
        i = 0
        while True:
            if not all(np.diag(ret_R1@A[i].T + ret_t1) == B[i]):
                B = np.delete(B, i, axis=0)
                A = np.delete(A, i, axis=0)            
            else:
                i += 1
            if len(B) == i:
                break
        if len(B) > 3:
            ret_R1, ret_t1 = rigid_transform_3D(A[:].T, B[:].T)
            ret_R2, ret_t2 = rigid_transform_3D(B[:].T, A[:].T)

            correlation_map[x] += [y]

            transformation_map[(x, y)] = ret_R1, ret_t1
            transformation_map[(y, x)] = ret_R2, ret_t2

            # diagnostic data
            all_examples.append(
                [np.asarray(list_B_dist)[sorting_B][0], np.asarray(list_A_dist)[sorting_A][0]])
            all_examples2.append([A, B])
            all_xy.append((x,y))
            all_transforms.append((ret_R1, ret_t1))

# we only investigated relationships in a single direction:
# n -> m but we didn't track m -> n to save some proc time
# so now we copy this relationship for m -> n
for k in range(max(correlation_map.values())[0] + 1):
    if k not in correlation_map.keys():
        correlation_map[k] = []

for k, v in correlation_map.items():
    for vk in v:
        if k not in correlation_map[vk]:
            correlation_map[vk] += [k]

Wall time: 2.03 s


In [50]:
from collections import Counter


# for each scanner, investigate its known relationships with other scanners
# and find the relationship path we need to follow to end up at scanner 0
# e.g.: [34, 15, 6, 13, 21, 2, 0]
# NOTE: below function is not fully correct but it somehow works xD 

def build_correlation_graph(curr_path, correlation_map):
    final_path = curr_path
    last_position = curr_path[-1]
    if last_position == 0:
        return final_path
    possible_moves = correlation_map[last_position]
    counter = Counter(curr_path)
    possible_moves = [m for m in possible_moves]
    for pos_move in sorted(possible_moves):
        temp_path = curr_path.copy() + [pos_move]
        if pos_move == 0 or counter[pos_move] == 2:
            return temp_path
        final_path = build_correlation_graph(temp_path, correlation_map)
        if final_path[-1] == 0:
            return final_path
    return final_path

transformation_paths = []
for i in sorted(correlation_map.keys(), reverse=True):
    curr_path = [i]
    transformation_paths.append(build_correlation_graph(
        curr_path, correlation_map))

In [51]:
# calculate scanner positions in relationship to scanner 0
# by applying matrix transformations based on transformations path
scanner_coords = []
for path in transformation_paths:
    temp_coords = np.asarray([0,0,0])
    if len(path) > 1:
        for i in range(1, len(path)):
            ret_R, ret_t = transformation_map[(path[i-1], path[i])]
            temp_coords = np.diag((ret_R@temp_coords) + ret_t)
        scanner_coords.append(temp_coords)

scanner_coords = [[round(c) for c in scanner] for scanner in scanner_coords]
scanner_coords = np.asarray(scanner_coords, dtype=int)

x, y = generate_xy(len(scanner_coords))
out_arr = np.abs(scanner_coords[x] - scanner_coords[y])
out_arr = out_arr.sum(axis=1)

max(out_arr)

13348

# Day 20
[[back to navigation]](#Navigation)  

Task details: https://adventofcode.com/2021/day/20

In [52]:
import numpy as np

with open("data/day20-input.txt", 'r') as file:
    inputs = file.read()
    inputs = inputs.split('\n\n')
    decoder = inputs[0]
    decoder = decoder.replace('\n','')
    input_im = inputs[1]

In [53]:
pix_dict = {'.': 0, '#': 1}
decoder = [pix_dict[pix] for pix in decoder]
input_im = np.asarray([[pix_dict[pix] for pix in l] for l in input_im.split('\n')])

In [54]:
input_im.shape

(100, 100)

In [55]:
input_im.sum()

5010

In [56]:
def enhance(input_im, decoder):
    sum_border = (
            input_im[0,:].sum() + input_im[:,0].sum()
            + input_im[:,-1].sum() + input_im[-1,:].sum()
        )
    pad = 3
    if sum_border == 0:
        input_im = np.pad(input_im, pad)
    else:
        input_im = np.pad(input_im, pad, constant_values=1)
    # create canvas
    canvas = np.zeros_like(input_im)
    # sliding window
    for i in range(1, input_im.shape[0]-1):
        for j in range(1, input_im.shape[1]-1):
            window = input_im[i-1:i+2,j-1:j+2]
            binary = ''.join([str(b) for b in window.ravel()])
            idx = int(binary, 2)
            canvas[i,j] = decoder[idx]
    return canvas[2:input_im.shape[0]-2, 2:input_im.shape[0]-2]

In [57]:
input_im = np.pad(input_im, 1)
for i in range(1, 51):
    input_im = enhance(input_im, decoder)
    if i == 2:
        print("Part one answer: ", input_im.sum())
    if i == 50:
        print("Part two answer: ", input_im.sum())

Part one answer:  5479
Part two answer:  19012
